<img src="assets/bootcamp.png">

# **LTI&trade; Advantage** bootcamp notebook for Tool

claude.vervoort@gmail.com

## Introduction

The notebook shows how to interact with the LTI Advantage ecosystem from a tool implementer viewpoint. It interacts with an actual test server which has been built as a platform simulator to support this notebook. 


In [155]:
# This notebook queries an actual tool platform test server. It needs its location:
platform_url='http://localhost:5000'

## Setup

Here we just import the libraries that will be needed in this notebook, define some utility functions and constants.

In [156]:
import requests
import json
import jwt
import base64
from time import time
from cryptography.hazmat.primitives.asymmetric.rsa import RSAPublicNumbers
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.serialization import Encoding, PublicFormat
from IPython.display import display, Markdown, HTML, Javascript

def decode_int(b64value):
    return int.from_bytes(base64.urlsafe_b64decode(b64value), byteorder='big')

# for concise code, return full claim prefixed by ims
def fc(claim):
    return "http://imsglobal.org/lti/{0}".format(claim)

def md(mdt):
    display(Markdown(mdt))
    
md_buffer = ''

def mdb(mdt=None):
    global md_buffer
    if mdt:
        md_buffer = md_buffer + '\n' + mdt
    else:
        md(md_buffer)
        md_buffer = ''


First we need to get a new tool deployment from the server for this notebook instance to use.
Each tool must have a client_id and a private key that will be used to interact with the platform services and send messages back to the platform.
It also needs the keyset URL that exposes the platform public keys needed to validate the incoming messages.

While this information is required for each tool, how it is obtained by the tool is NOT currently part of the LTI specifications.

In [157]:
r = requests.get(platform_url + '/newtool')
tool_info = json.loads(r.text)

md('### Tool information')
md('Here is the tool information generated for this notebook. It is stored in ```tool_info``` variable.')
md('```json\n'+ json.dumps(tool_info, indent=4)+'```')

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

### Getting the public keys from the platform

In order to validate the various messages we will receive from the platform, we need to get the key sets. We'll also transform the keys to the `PEM` format that is used by the `jwt` module to decode the messages.

The test platform exposes its keys in a keyset format at a well-know location (.well-known/jwks.json). Other platform might just communicate the keyset url as part of the tool deployment information.

In [154]:
keyset = json.loads(requests.get(platform_url + '/.well-known/jwks.json').text)

md('#### Platform keyset')
md('```json\n'+ json.dumps(keyset, indent=4)+'```')

platform_keys = {}

# let's transform as a map for ease of use, and just the PEM because this is what is used by JWT lib
for key in keyset['keys']:
    public_key = RSAPublicNumbers(decode_int(key['e']), decode_int(key['n'])).public_key(default_backend())
    pem = public_key.public_bytes(Encoding.PEM, PublicFormat.SubjectPublicKeyInfo)
    platform_keys[key['kid']] = pem
    


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

## Deep Linking - Creating a Link
This section will use the deep linking specification to create a Resource Link to the platform. That resource link will be gradable and used in the following sections of that notebook.

[Deep Linking](https://www.imsglobal.org/specs/lticiv1p0) is a **UI flow**, it is an important piece that is sometimes missed on 1st reading. The user is redirected from the platform to the tool to pick or create one or multiple piece of content (often, LTI links), and the the tool redirects the UI back to the tool with the actual selection (or an empty payload if nothing was picked or created).

So there are 2 messages:

1. `LTIDeepLinkingRequest` from the platform to the tool to start the picking/create session. This is a typical platform launch that contains the context and the user information, and what kind of content items may be created in this flow (for example, this flow might indicate it only wants LTI links).
1. `LTIDeepLinkingResponse` from the tool back to the plaform using the `content_item_return_url` provided in the request.

Once a tool is added to a course, usually the first launch from the platform will be a Deep Linking request.

<img src="assets/lti_advantage_deeplinking.png" width="60%">

### Setup: Getting a Deep Linking Request

This notebook is not a tool actually launched by the platform, so the test platform as way to give us the token that it would include in an actual HTTP POST request, so we can build a mock POST request including the parameter `post_data`.

In [142]:
r = requests.get("{}/tool/{}/cisr".format(platform_url, tool_info['client_id']))

post_data = {
    'id_token':r.text
}

md('`id_token='+ r.text+'`')

`id_token=eyJ0eXAiOiJKV1QiLCJhbGciOiJSUzI1NiIsImtpZCI6IjE1MTg2NjUwMDlfMSJ9.eyJub25jZSI6ImI4MjBiNjM0LTExZmYtMTFlOC05NDY5LWM0OGU4ZmZiNzg1NyIsImh0dHA6Ly9pbXNnbG9iYWwub3JnL2x0aS92ZXJzaW9uIjoiMS4zLjAiLCJodHRwOi8vaW1zZ2xvYmFsLm9yZy9sdGkvdG9vbF9wbGF0Zm9ybSI6eyJndWlkIjoibHRpYmNfYXRfMTUxODY2NTAxMiIsIm5hbWUiOiJMVEkgQm9vdGNhbXAgUGxhdGZvcm0ifSwiaHR0cDovL2ltc2dsb2JhbC5vcmcvbHRpL2FncyI6eyJzY29wZSI6WyJodHRwOi8vaW1zZ2xvYmFsLm9yZy9hZ3MvbGluZWl0ZW0iLCJodHRwOi8vaW1zZ2xvYmFsLm9yZy9hZ3MvcmVzdWx0L3JlYWQiLCJodHRwOi8vaW1zZ2xvYmFsLm9yZy9hZ3Mvc2NvcmUvcHVibGlzaCJdLCJsaW5laXRlbXMiOiJodHRwOi8vbG9jYWxob3N0OjUwMDAvYjY2YTBhNWMtMTFmZi0xMWU4LTk0NjktYzQ4ZThmZmI3ODU3L2xpbmVpdGVtcyJ9LCJpc3MiOiJodHRwOi8vbG9jYWxob3N0OjUwMDAiLCJodHRwOi8vaW1zZ2xvYmFsLm9yZy9sdGkvZGVwbG95bWVudF9pZCI6ImRlcGxveW1lbnRfMCIsImVtYWlsIjoiS2FyYS5UaHJhY2VAZXhhbXBsZS5jb20iLCJmYW1pbHlfbmFtZSI6IlRocmFjZSIsImV4cCI6MTUxODY2NTEyMywiaHR0cDovL2ltc2dsb2JhbC5vcmcvbHRpL2xhdW5jaF9wcmVzZW50YXRpb24iOnsiZG9jdW1lbnRfdGFyZ2V0IjoiaWZyYW1lIiwicmV0dXJuX3VybCI6Imh0dHA6Ly9sb2NhbGhvc3Q6NTAwMC90b29sL2I2NmEwYTVjLTExZmYtMTFlOC05NDY5LWM0OGU4ZmZiNzg1Ny9jaXIifSwiaHR0cDovL2ltc2dsb2JhbC5vcmcvbHRpL2RlZXBfbGlua2luZ19yZXF1ZXN0Ijp7ImF1dG9fY3JlYXRlIjp0cnVlLCJhY2NlcHRfbXVsdGlwbGUiOnRydWUsImFjY2VwdF9tZWRpYV90eXBlcyI6WyJhcHBsaWNhdGlvbi92bmQuaW1zLmx0aS52MS5sdGlsaW5rIl0sImFjY2VwdF9wcmVzZW50YXRpb25fZG9jdW1lbnRfdGFyZ2V0cyI6WyJpZnJhbWUiLCJ3aW5kb3ciXSwiZGF0YSI6Im9wPTMyMSZ2PTQ0In0sImlhdCI6MTUxODY2NTA2MywiaHR0cDovL2ltc2dsb2JhbC5vcmcvbHRpL3Rva2VuIjoiaHR0cDovL2xvY2FsaG9zdDo1MDAwL2F1dGgvdG9rZW4iLCJhdWQiOiIwIiwiaHR0cDovL2ltc2dsb2JhbC5vcmcvbHRpL21lc3NhZ2VfdHlwZSI6IkxUSURlZXBMaW5raW5nUmVxdWVzdCIsImdpdmVuX25hbWUiOiJLYXJhIiwibmFtZSI6IkthcmEgVGhyYWNlIiwic3ViIjoiTFRJQkNVXzEzIiwiaHR0cDovL2ltc2dsb2JhbC5vcmcvbHRpL2NvbnRleHQiOnsiaWQiOiJiNjZhMGE1Yy0xMWZmLTExZTgtOTQ2OS1jNDhlOGZmYjc4NTciLCJsYWJlbCI6IkxUSSBCb290Y2FtcCBDb3Vyc2UiLCJ0eXBlIjpbIkNvdXJzZVNlY3Rpb24iXSwidGl0bGUiOiJMVEkgQm9vdGNhbXAgQ291cnNlIn0sImh0dHA6Ly9pbXNnbG9iYWwub3JnL2x0aS9yb2xlcyI6WyJodHRwOi8vcHVybC5pbXNnbG9iYWwub3JnL3ZvY2FiL2xpcy92Mi9tZW1iZXJzaGlwI0luc3RydWN0b3IiXX0.HPbKFddZxO3YCkhWNaPmkSpxLLy6XABABIv9KsXf-xr4R8t4UQikQltAFSnWPrTXWXmTbLOcV8FFKkVQXAnoIzSdAnsNyYmFxzNg8ZgRVWLEsNBT67wC9sve5s1mJYPqAA1T0SoPPN9tpPgNAZ1WEBq1JjQco6A2CEDdGVBK7TtV1taj688QahPYLb2RaQHGeQ3KFToeLz6zZLdO7QmQ9ZrSUKYwtNW0tMaPn0wJZQdpVphD65bdTM9vbfRnRkPWoDy1Qrr3G52iPlu8qR07BLn7s6Jg9_D4v2IUKSQMAnsyMXrbG6CgtgfiX0AK3KmNpPQMNDp07ID3vX2a6yKUCw`

### Task 1: Verify the JWT is properly signed

The first thing before to display to the user the picker/authoring interface to create the link is to validate this request is properly signed. This is done by decoding the JWT using public key from the platform.


In [143]:
# Let's get the kid so we can get the proper public key

# should check ISS first to lookup keyset

encoded_jwt = post_data['id_token']
jwt_headers = jwt.get_unverified_header(encoded_jwt)

content_item_message = jwt.decode(encoded_jwt, 
                                  platform_keys[jwt_headers['kid']], 
                                  jwt_headers['alg'],
                                  audience = tool_info['client_id'])

md('#### Message properly signed! Decoded ContentItemSelectionRequest message:')
md('```json\n'+ json.dumps(content_item_message, indent=4)+'```')


#### Message properly signed! Decoded ContentItemSelectionRequest message:

```json
{
    "http://imsglobal.org/lti/context": {
        "id": "b66a0a5c-11ff-11e8-9469-c48e8ffb7857",
        "label": "LTI Bootcamp Course",
        "type": [
            "CourseSection"
        ],
        "title": "LTI Bootcamp Course"
    },
    "http://imsglobal.org/lti/ags": {
        "lineitems": "http://localhost:5000/b66a0a5c-11ff-11e8-9469-c48e8ffb7857/lineitems",
        "scope": [
            "http://imsglobal.org/ags/lineitem",
            "http://imsglobal.org/ags/result/read",
            "http://imsglobal.org/ags/score/publish"
        ]
    },
    "http://imsglobal.org/lti/version": "1.3.0",
    "name": "Kara Thrace",
    "aud": "0",
    "exp": 1518665123,
    "nonce": "b820b634-11ff-11e8-9469-c48e8ffb7857",
    "family_name": "Thrace",
    "http://imsglobal.org/lti/launch_presentation": {
        "document_target": "iframe",
        "return_url": "http://localhost:5000/tool/b66a0a5c-11ff-11e8-9469-c48e8ffb7857/cir"
    },
    "iss": "http://localhost:5000",
    "http://imsglobal.org/lti/roles": [
        "http://purl.imsglobal.org/vocab/lis/v2/membership#Instructor"
    ],
    "email": "Kara.Thrace@example.com",
    "http://imsglobal.org/lti/deployment_id": "deployment_0",
    "http://imsglobal.org/lti/tool_platform": {
        "guid": "ltibc_at_1518665012",
        "name": "LTI Bootcamp Platform"
    },
    "iat": 1518665063,
    "http://imsglobal.org/lti/deep_linking_request": {
        "accept_multiple": true,
        "accept_presentation_document_targets": [
            "iframe",
            "window"
        ],
        "data": "op=321&v=44",
        "auto_create": true,
        "accept_media_types": [
            "application/vnd.ims.lti.v1.ltilink"
        ]
    },
    "http://imsglobal.org/lti/message_type": "LTIDeepLinkingRequest",
    "sub": "LTIBCU_13",
    "given_name": "Kara",
    "http://imsglobal.org/lti/token": "http://localhost:5000/auth/token"
}```

### Task 2: extract the information needed to render the selector/authoring UI

If this is the first launch for the user or the course, as a tool you may prompt the user for setup information, including account linking or course setup. Ultimately the user will see the authoring or picking interface that will allow her to create or select the content items to be added to the course.

Some key attributes of the `ContentItemSelectionRequest` will drive the experience:

In [144]:
# fc(claim) prefix the claim with http://imsglobal.org/lti/
mdb('1. What is the current course id? {0}'.format(content_item_message[fc('context')]['id']))
mdb('1. What is the current user id? {0}'.format(content_item_message['sub']))
is_instructor = len(list(filter(lambda x: 'instructor' in x.lower(), content_item_message[fc('roles')])))>0
mdb('1. Is this user an instructor? {0}'.format(is_instructor))
deep_linking_claim = content_item_message[fc('deep_linking_request')]
mdb('1. What kind of content item can be created? {0}'.format(deep_linking_claim['accept_media_types']))
mdb('1. Can I return more than one items to be added? {0}'.format(deep_linking_claim['accept_multiple']))
mdb('1. Will the user be prompted before to actually save the items? {0}'.format(not deep_linking_claim['auto_create']))
deep_linking_return_url = content_item_message[fc('launch_presentation')]['return_url']
mdb('1. Where should I redirect the browser too when done? {0}'.format(deep_linking_return_url))
mdb('1. Is there any data I must pass back to platform when I return? {0}'.format('data' in deep_linking_claim))
mdb()


1. What is the current course id? b66a0a5c-11ff-11e8-9469-c48e8ffb7857
1. What is the current user id? LTIBCU_13
1. Is this user an instructor? True
1. What kind of content item can be created? ['application/vnd.ims.lti.v1.ltilink']
1. Can I return more than one items to be added? True
1. Will the user be prompted before to actually save the items? False
1. Where should I redirect the browser too when done? http://localhost:5000/tool/b66a0a5c-11ff-11e8-9469-c48e8ffb7857/cir
1. Is there any data I must pass back to platform when I return? True

### Task 3: building the response token

After the end of the interaction, so user is sent back to the platform throught a browser redirection using an HTTP POST containing the JWT `ContentItemResponse` message. In this case, we will return 2 LTI links, one being graded (since the request supports multiple content items).

Here we're creating the actual response token.

In [145]:
## First let's create our 2 content items
## Note that the URLs are phony as for now there is now way to launch in the notebook anyway...
simple_link =  {
      "mediaType": "application/vnd.ims.lti.v1.ltilink",
      "url": "http://lti.bootcamp/item111",
      "presentationDocumentTarget": "iframe",
      "title": "A simple content item",
      "text": "Some long text",
      "icon": {
        "url": "http://lti.example.com/image.jpg",
        "width": 100,
        "height": 100
      },
      "custom": {
        "lab": "sim4e"
      }
}

assignment_link =  {
      "mediaType": "application/vnd.ims.lti.v1.ltilink",
      "url": "http://lti.bootcamp/item111",
      "presentationDocumentTarget": "iframe",
      "title": "An assignment",
      "text": "Chemical lab sim",
      "icon": {
        "url": "http://lti.example.com/image.jpg",
        "width": 100,
        "height": 100
      },
      "custom": {
        "lab": "sim3a",
        "level": "easy"
      },
      "lineItem": {
        "scoreMaximum": 34,
        "label": "Chemical lab sim",
        "resourceId": "sim3a",
        "tag": "final_grade"
      }
}

now = int(time())

deep_linking_response = {
    "iss": tool_info['client_id'],
    "aud": content_item_message['iss'],
    "exp": now + 60,
    "iat": now,
    "http://imsglobal.org/lti/message_type": "DeepLinkingResponse",
    "http://imsglobal.org/lti/version": "1.3.0",
    "http://imsglobal.org/lti/content_items": [
        simple_link, assignment_link
    ]
    
}

### Task 4: build the signed JWT

In [146]:
deep_linking_response_token = jwt.encode(deep_linking_response, tool_info['webkeyPem'], 'RS256').decode()

print(deep_linking_response_token)

eyJ0eXAiOiJKV1QiLCJhbGciOiJSUzI1NiJ9.eyJodHRwOi8vaW1zZ2xvYmFsLm9yZy9sdGkvdmVyc2lvbiI6IjEuMy4wIiwiaWF0IjoxNTE4NjY1MDY4LCJodHRwOi8vaW1zZ2xvYmFsLm9yZy9sdGkvY29udGVudF9pdGVtcyI6W3sidXJsIjoiaHR0cDovL2x0aS5ib290Y2FtcC9pdGVtMTExIiwibWVkaWFUeXBlIjoiYXBwbGljYXRpb24vdm5kLmltcy5sdGkudjEubHRpbGluayIsInRpdGxlIjoiQSBzaW1wbGUgY29udGVudCBpdGVtIiwidGV4dCI6IlNvbWUgbG9uZyB0ZXh0IiwiY3VzdG9tIjp7ImxhYiI6InNpbTRlIn0sInByZXNlbnRhdGlvbkRvY3VtZW50VGFyZ2V0IjoiaWZyYW1lIiwiaWNvbiI6eyJ3aWR0aCI6MTAwLCJ1cmwiOiJodHRwOi8vbHRpLmV4YW1wbGUuY29tL2ltYWdlLmpwZyIsImhlaWdodCI6MTAwfX0seyJ1cmwiOiJodHRwOi8vbHRpLmJvb3RjYW1wL2l0ZW0xMTEiLCJwcmVzZW50YXRpb25Eb2N1bWVudFRhcmdldCI6ImlmcmFtZSIsInRpdGxlIjoiQW4gYXNzaWdubWVudCIsInRleHQiOiJDaGVtaWNhbCBsYWIgc2ltIiwiY3VzdG9tIjp7ImxldmVsIjoiZWFzeSIsImxhYiI6InNpbTNhIn0sImxpbmVJdGVtIjp7InNjb3JlTWF4aW11bSI6MzQsInJlc291cmNlSWQiOiJzaW0zYSIsInRhZyI6ImZpbmFsX2dyYWRlIiwibGFiZWwiOiJDaGVtaWNhbCBsYWIgc2ltIn0sIm1lZGlhVHlwZSI6ImFwcGxpY2F0aW9uL3ZuZC5pbXMubHRpLnYxLmx0aWxpbmsiLCJpY29uIjp7IndpZHRoIjoxMDAsInVybCI

### Task 5: redirect the user back to the platform with the content item selection

Now that we the response token, let's do the actual HTML POST redirection to the platform. Note that because the platform supports `autocreate` there will be no prompt. The 2 items will be added directly to the course.


In [147]:
# Let's start by adding the JWS security claims
content_item_response = {
    'iss': tool_info['client_id'] ,
    'aud': content_item_message['iss']
}

autosubmit_js = """
                var ltiForm = $('<form>');                
                ltiForm.attr('action', '{url}');
                ltiForm.attr('method', 'POST');
                ltiForm.attr('target', 'deeplinking_frame');
                $('<input>').attr({{
                    type: 'hidden',
                    name: 'jws_token',
                    value: '{token}'
                }}).appendTo(ltiForm);
                $('#deeplinking_frame').before(ltiForm);
                ltiForm.submit();
                ltiForm.remove();
                """

autosubmit_js = autosubmit_js.format(url=deep_linking_return_url, token=deep_linking_response_token)

display(HTML('<iframe id="deeplinking_frame" name="deeplinking_frame" style="height: 300px; width:100%"></iframe>'))
display(Javascript(data=autosubmit_js, 
                   lib="https://code.jquery.com/jquery-3.3.1.min.js"))

<IPython.core.display.Javascript object>

## Student Resource Link launch

Now that we have created resource links, let's handle a student launch from one of them. We're going to use a resource link with a **coupled** line item, so that we can use it to send a score back to the platform.

### Setup

The first thing we need, as with deep linking, is to get from the test platform the launch token which an actual tool would get in an actual HTML Form Post.

In [150]:
# select an id from the ones displayed in the course platform in the IFrame above
# if not specified the platform will pick a resource to use

resource_link_id = 'bc5bec65-11ff-11e8-9469-c48e8ffb7857'
context_id = content_item_message['http://imsglobal.org/lti/context']['id']

r = requests.get("{}/tool/{}/context/{}/studentlaunch?rlid={}".format(platform_url, 
                                                           tool_info['client_id'], 
                                                           context_id, 
                                                           resource_link_id))

post_data = {
    'id_token':r.text
}

md('`id_token='+ r.text+'`')

`id_token=<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN"
  "http://www.w3.org/TR/html4/loose.dtd">
<html>
  <head>
    <title>AttributeError: 'LineItem' object has no attribute 'id' // Werkzeug Debugger</title>
    <link rel="stylesheet" href="?__debugger__=yes&amp;cmd=resource&amp;f=style.css"
        type="text/css">
    <!-- We need to make sure this has a favicon so that the debugger does
         not by accident trigger a request to /favicon.ico which might
         change the application state. -->
    <link rel="shortcut icon"
        href="?__debugger__=yes&amp;cmd=resource&amp;f=console.png">
    <script src="?__debugger__=yes&amp;cmd=resource&amp;f=jquery.js"></script>
    <script src="?__debugger__=yes&amp;cmd=resource&amp;f=debugger.js"></script>
    <script type="text/javascript">
      var TRACEBACK = 140714606927656,
          CONSOLE_MODE = false,
          EVALEX = true,
          EVALEX_TRUSTED = false,
          SECRET = "sbxeTaweHc39TMEVsQ10";
    </script>
  </head>
  <body style="background-color: #fff">
    <div class="debugger">
<h1>builtins.AttributeError</h1>
<div class="detail">
  <p class="errormsg">AttributeError: 'LineItem' object has no attribute 'id'</p>
</div>
<h2 class="traceback">Traceback <em>(most recent call last)</em></h2>
<div class="traceback">
  
  <ul><li><div class="frame" id="frame-140714606927824">
  <h4>File <cite class="filename">"/home/claude/.py3envs/lti_bootcamp/lib/python3.5/site-packages/flask/app.py"</cite>,
      line <em class="line">1997</em>,
      in <code class="function">__call__</code></h4>
  <div class="source"><pre class="line before"><span class="ws">                </span>error = None</pre>
<pre class="line before"><span class="ws">            </span>ctx.auto_pop(error)</pre>
<pre class="line before"><span class="ws"></span> </pre>
<pre class="line before"><span class="ws">    </span>def __call__(self, environ, start_response):</pre>
<pre class="line before"><span class="ws">        </span>&quot;&quot;&quot;Shortcut for :attr:`wsgi_app`.&quot;&quot;&quot;</pre>
<pre class="line current"><span class="ws">        </span>return self.wsgi_app(environ, start_response)</pre>
<pre class="line after"><span class="ws"></span> </pre>
<pre class="line after"><span class="ws">    </span>def __repr__(self):</pre>
<pre class="line after"><span class="ws">        </span>return '&lt;%s %r&gt;' % (</pre>
<pre class="line after"><span class="ws">            </span>self.__class__.__name__,</pre>
<pre class="line after"><span class="ws">            </span>self.name,</pre></div>
</div>

<li><div class="frame" id="frame-140714606927768">
  <h4>File <cite class="filename">"/home/claude/.py3envs/lti_bootcamp/lib/python3.5/site-packages/flask/app.py"</cite>,
      line <em class="line">1985</em>,
      in <code class="function">wsgi_app</code></h4>
  <div class="source"><pre class="line before"><span class="ws">        </span>try:</pre>
<pre class="line before"><span class="ws">            </span>try:</pre>
<pre class="line before"><span class="ws">                </span>response = self.full_dispatch_request()</pre>
<pre class="line before"><span class="ws">            </span>except Exception as e:</pre>
<pre class="line before"><span class="ws">                </span>error = e</pre>
<pre class="line current"><span class="ws">                </span>response = self.handle_exception(e)</pre>
<pre class="line after"><span class="ws">            </span>except:</pre>
<pre class="line after"><span class="ws">                </span>error = sys.exc_info()[1]</pre>
<pre class="line after"><span class="ws">                </span>raise</pre>
<pre class="line after"><span class="ws">            </span>return response(environ, start_response)</pre>
<pre class="line after"><span class="ws">        </span>finally:</pre></div>
</div>

<li><div class="frame" id="frame-140714606760232">
  <h4>File <cite class="filename">"/home/claude/.py3envs/lti_bootcamp/lib/python3.5/site-packages/flask/app.py"</cite>,
      line <em class="line">1540</em>,
      in <code class="function">handle_exception</code></h4>
  <div class="source"><pre class="line before"><span class="ws">            </span># if we want to repropagate the exception, we can attempt to</pre>
<pre class="line before"><span class="ws">            </span># raise it with the whole traceback in case we can do that</pre>
<pre class="line before"><span class="ws">            </span># (the function was actually called from the except part)</pre>
<pre class="line before"><span class="ws">            </span># otherwise, we just raise the error again</pre>
<pre class="line before"><span class="ws">            </span>if exc_value is e:</pre>
<pre class="line current"><span class="ws">                </span>reraise(exc_type, exc_value, tb)</pre>
<pre class="line after"><span class="ws">            </span>else:</pre>
<pre class="line after"><span class="ws">                </span>raise e</pre>
<pre class="line after"><span class="ws"></span> </pre>
<pre class="line after"><span class="ws">        </span>self.log_exception((exc_type, exc_value, tb))</pre>
<pre class="line after"><span class="ws">        </span>if handler is None:</pre></div>
</div>

<li><div class="frame" id="frame-140714606760288">
  <h4>File <cite class="filename">"/home/claude/.py3envs/lti_bootcamp/lib/python3.5/site-packages/flask/_compat.py"</cite>,
      line <em class="line">33</em>,
      in <code class="function">reraise</code></h4>
  <div class="source"><pre class="line before"><span class="ws">    </span>from io import StringIO</pre>
<pre class="line before"><span class="ws"></span> </pre>
<pre class="line before"><span class="ws">    </span>def reraise(tp, value, tb=None):</pre>
<pre class="line before"><span class="ws">        </span>if value.__traceback__ is not tb:</pre>
<pre class="line before"><span class="ws">            </span>raise value.with_traceback(tb)</pre>
<pre class="line current"><span class="ws">        </span>raise value</pre>
<pre class="line after"><span class="ws"></span> </pre>
<pre class="line after"><span class="ws">    </span>implements_to_string = _identity</pre>
<pre class="line after"><span class="ws"></span> </pre>
<pre class="line after"><span class="ws"></span>else:</pre>
<pre class="line after"><span class="ws">    </span>text_type = unicode</pre></div>
</div>

<li><div class="frame" id="frame-140714606760120">
  <h4>File <cite class="filename">"/home/claude/.py3envs/lti_bootcamp/lib/python3.5/site-packages/flask/app.py"</cite>,
      line <em class="line">1982</em>,
      in <code class="function">wsgi_app</code></h4>
  <div class="source"><pre class="line before"><span class="ws">        </span>ctx = self.request_context(environ)</pre>
<pre class="line before"><span class="ws">        </span>ctx.push()</pre>
<pre class="line before"><span class="ws">        </span>error = None</pre>
<pre class="line before"><span class="ws">        </span>try:</pre>
<pre class="line before"><span class="ws">            </span>try:</pre>
<pre class="line current"><span class="ws">                </span>response = self.full_dispatch_request()</pre>
<pre class="line after"><span class="ws">            </span>except Exception as e:</pre>
<pre class="line after"><span class="ws">                </span>error = e</pre>
<pre class="line after"><span class="ws">                </span>response = self.handle_exception(e)</pre>
<pre class="line after"><span class="ws">            </span>except:</pre>
<pre class="line after"><span class="ws">                </span>error = sys.exc_info()[1]</pre></div>
</div>

<li><div class="frame" id="frame-140714606760400">
  <h4>File <cite class="filename">"/home/claude/.py3envs/lti_bootcamp/lib/python3.5/site-packages/flask/app.py"</cite>,
      line <em class="line">1614</em>,
      in <code class="function">full_dispatch_request</code></h4>
  <div class="source"><pre class="line before"><span class="ws">            </span>request_started.send(self)</pre>
<pre class="line before"><span class="ws">            </span>rv = self.preprocess_request()</pre>
<pre class="line before"><span class="ws">            </span>if rv is None:</pre>
<pre class="line before"><span class="ws">                </span>rv = self.dispatch_request()</pre>
<pre class="line before"><span class="ws">        </span>except Exception as e:</pre>
<pre class="line current"><span class="ws">            </span>rv = self.handle_user_exception(e)</pre>
<pre class="line after"><span class="ws">        </span>return self.finalize_request(rv)</pre>
<pre class="line after"><span class="ws"></span> </pre>
<pre class="line after"><span class="ws">    </span>def finalize_request(self, rv, from_error_handler=False):</pre>
<pre class="line after"><span class="ws">        </span>&quot;&quot;&quot;Given the return value from a view function this finalizes</pre>
<pre class="line after"><span class="ws">        </span>the request by converting it into a response and invoking the</pre></div>
</div>

<li><div class="frame" id="frame-140714606760456">
  <h4>File <cite class="filename">"/home/claude/.py3envs/lti_bootcamp/lib/python3.5/site-packages/flask/app.py"</cite>,
      line <em class="line">1517</em>,
      in <code class="function">handle_user_exception</code></h4>
  <div class="source"><pre class="line before"><span class="ws">            </span>return self.handle_http_exception(e)</pre>
<pre class="line before"><span class="ws"></span> </pre>
<pre class="line before"><span class="ws">        </span>handler = self._find_error_handler(e)</pre>
<pre class="line before"><span class="ws"></span> </pre>
<pre class="line before"><span class="ws">        </span>if handler is None:</pre>
<pre class="line current"><span class="ws">            </span>reraise(exc_type, exc_value, tb)</pre>
<pre class="line after"><span class="ws">        </span>return handler(e)</pre>
<pre class="line after"><span class="ws"></span> </pre>
<pre class="line after"><span class="ws">    </span>def handle_exception(self, e):</pre>
<pre class="line after"><span class="ws">        </span>&quot;&quot;&quot;Default exception handling that kicks in when an exception</pre>
<pre class="line after"><span class="ws">        </span>occurs that is not caught.  In debug mode the exception will</pre></div>
</div>

<li><div class="frame" id="frame-140714606760512">
  <h4>File <cite class="filename">"/home/claude/.py3envs/lti_bootcamp/lib/python3.5/site-packages/flask/_compat.py"</cite>,
      line <em class="line">33</em>,
      in <code class="function">reraise</code></h4>
  <div class="source"><pre class="line before"><span class="ws">    </span>from io import StringIO</pre>
<pre class="line before"><span class="ws"></span> </pre>
<pre class="line before"><span class="ws">    </span>def reraise(tp, value, tb=None):</pre>
<pre class="line before"><span class="ws">        </span>if value.__traceback__ is not tb:</pre>
<pre class="line before"><span class="ws">            </span>raise value.with_traceback(tb)</pre>
<pre class="line current"><span class="ws">        </span>raise value</pre>
<pre class="line after"><span class="ws"></span> </pre>
<pre class="line after"><span class="ws">    </span>implements_to_string = _identity</pre>
<pre class="line after"><span class="ws"></span> </pre>
<pre class="line after"><span class="ws"></span>else:</pre>
<pre class="line after"><span class="ws">    </span>text_type = unicode</pre></div>
</div>

<li><div class="frame" id="frame-140714606760176">
  <h4>File <cite class="filename">"/home/claude/.py3envs/lti_bootcamp/lib/python3.5/site-packages/flask/app.py"</cite>,
      line <em class="line">1612</em>,
      in <code class="function">full_dispatch_request</code></h4>
  <div class="source"><pre class="line before"><span class="ws">        </span>self.try_trigger_before_first_request_functions()</pre>
<pre class="line before"><span class="ws">        </span>try:</pre>
<pre class="line before"><span class="ws">            </span>request_started.send(self)</pre>
<pre class="line before"><span class="ws">            </span>rv = self.preprocess_request()</pre>
<pre class="line before"><span class="ws">            </span>if rv is None:</pre>
<pre class="line current"><span class="ws">                </span>rv = self.dispatch_request()</pre>
<pre class="line after"><span class="ws">        </span>except Exception as e:</pre>
<pre class="line after"><span class="ws">            </span>rv = self.handle_user_exception(e)</pre>
<pre class="line after"><span class="ws">        </span>return self.finalize_request(rv)</pre>
<pre class="line after"><span class="ws"></span> </pre>
<pre class="line after"><span class="ws">    </span>def finalize_request(self, rv, from_error_handler=False):</pre></div>
</div>

<li><div class="frame" id="frame-140714606760624">
  <h4>File <cite class="filename">"/home/claude/.py3envs/lti_bootcamp/lib/python3.5/site-packages/flask/app.py"</cite>,
      line <em class="line">1598</em>,
      in <code class="function">dispatch_request</code></h4>
  <div class="source"><pre class="line before"><span class="ws">        </span># request came with the OPTIONS method, reply automatically</pre>
<pre class="line before"><span class="ws">        </span>if getattr(rule, 'provide_automatic_options', False) \</pre>
<pre class="line before"><span class="ws">           </span>and req.method == 'OPTIONS':</pre>
<pre class="line before"><span class="ws">            </span>return self.make_default_options_response()</pre>
<pre class="line before"><span class="ws">        </span># otherwise dispatch to the handler for that endpoint</pre>
<pre class="line current"><span class="ws">        </span>return self.view_functions[rule.endpoint](**req.view_args)</pre>
<pre class="line after"><span class="ws"></span> </pre>
<pre class="line after"><span class="ws">    </span>def full_dispatch_request(self):</pre>
<pre class="line after"><span class="ws">        </span>&quot;&quot;&quot;Dispatches the request and on top of that performs request</pre>
<pre class="line after"><span class="ws">        </span>pre and postprocessing as well as HTTP exception catching and</pre>
<pre class="line after"><span class="ws">        </span>error handling.</pre></div>
</div>

<li><div class="frame" id="frame-140714606760680">
  <h4>File <cite class="filename">"/home/claude/projects/lti_bootcamp/server/lti_platform.py"</cite>,
      line <em class="line">77</em>,
      in <code class="function">student_launch</code></h4>
  <div class="source"><pre class="line before"><span class="ws">                                            </span>course,</pre>
<pre class="line before"><span class="ws">                                            </span>course.roster.getOneStudent(),</pre>
<pre class="line before"><span class="ws">                                            </span>{},</pre>
<pre class="line before"><span class="ws">                                            </span>request.url_root,</pre>
<pre class="line before"><span class="ws">                                            </span>request_url=request.url_root,</pre>
<pre class="line current"><span class="ws">                                            </span>resource_link=resource_link)</pre>
<pre class="line after"><span class="ws"></span> </pre>
<pre class="line after"><span class="ws"></span> </pre>
<pre class="line after"><span class="ws"></span>@app.route(&quot;/course/&lt;course_id&gt;&quot;)</pre>
<pre class="line after"><span class="ws"></span>def show_course(course_id):</pre>
<pre class="line after"><span class="ws">    </span>return render_template('courseoutline.html', course=platform.get_course(course_id))</pre></div>
</div>

<li><div class="frame" id="frame-140714606760064">
  <h4>File <cite class="filename">"/home/claude/projects/lti_bootcamp/server/ltiplatform/ltiplatform_manager.py"</cite>,
      line <em class="line">53</em>,
      in <code class="function">token</code></h4>
  <div class="source"><pre class="line before"><span class="ws">        </span>message = course.addToMessage(message)</pre>
<pre class="line before"><span class="ws"></span> </pre>
<pre class="line before"><span class="ws">        </span>if resource_link:</pre>
<pre class="line before"><span class="ws">            </span>message = resource_link.addToMessage(message)</pre>
<pre class="line before"><span class="ws">            </span>if resource_link.lineitem:</pre>
<pre class="line current"><span class="ws">                </span>ags_claim['lineitem'] = '{0}/{1}/lineitems/{2}'.format(root_url, course.id, resource_link.lineitem.id)</pre>
<pre class="line after"><span class="ws"></span> </pre>
<pre class="line after"><span class="ws">        </span>message['http://imsglobal.org/lti/ags'] = ags_claim</pre>
<pre class="line after"><span class="ws">        </span>message = self.platform.addToMessage(message)</pre>
<pre class="line after"><span class="ws">        </span>return jwt.encode(message, privatekey, algorithm='RS256', headers={'kid':key[0]})</pre>
<pre class="line after"><span class="ws"></span> </pre></div>
</div>
</ul>
  <blockquote>AttributeError: 'LineItem' object has no attribute 'id'</blockquote>
</div>

<div class="plain">
  <form action="/?__debugger__=yes&amp;cmd=paste" method="post">
    <p>
      <input type="hidden" name="language" value="pytb">
      This is the Copy/Paste friendly version of the traceback.  <span
      class="pastemessage">You can also paste this traceback into
      a <a href="https://gist.github.com/">gist</a>:
      <input type="submit" value="create paste"></span>
    </p>
    <textarea cols="50" rows="10" name="code" readonly>Traceback (most recent call last):
  File &quot;/home/claude/.py3envs/lti_bootcamp/lib/python3.5/site-packages/flask/app.py&quot;, line 1997, in __call__
    return self.wsgi_app(environ, start_response)
  File &quot;/home/claude/.py3envs/lti_bootcamp/lib/python3.5/site-packages/flask/app.py&quot;, line 1985, in wsgi_app
    response = self.handle_exception(e)
  File &quot;/home/claude/.py3envs/lti_bootcamp/lib/python3.5/site-packages/flask/app.py&quot;, line 1540, in handle_exception
    reraise(exc_type, exc_value, tb)
  File &quot;/home/claude/.py3envs/lti_bootcamp/lib/python3.5/site-packages/flask/_compat.py&quot;, line 33, in reraise
    raise value
  File &quot;/home/claude/.py3envs/lti_bootcamp/lib/python3.5/site-packages/flask/app.py&quot;, line 1982, in wsgi_app
    response = self.full_dispatch_request()
  File &quot;/home/claude/.py3envs/lti_bootcamp/lib/python3.5/site-packages/flask/app.py&quot;, line 1614, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File &quot;/home/claude/.py3envs/lti_bootcamp/lib/python3.5/site-packages/flask/app.py&quot;, line 1517, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File &quot;/home/claude/.py3envs/lti_bootcamp/lib/python3.5/site-packages/flask/_compat.py&quot;, line 33, in reraise
    raise value
  File &quot;/home/claude/.py3envs/lti_bootcamp/lib/python3.5/site-packages/flask/app.py&quot;, line 1612, in full_dispatch_request
    rv = self.dispatch_request()
  File &quot;/home/claude/.py3envs/lti_bootcamp/lib/python3.5/site-packages/flask/app.py&quot;, line 1598, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File &quot;/home/claude/projects/lti_bootcamp/server/lti_platform.py&quot;, line 77, in student_launch
    resource_link=resource_link)
  File &quot;/home/claude/projects/lti_bootcamp/server/ltiplatform/ltiplatform_manager.py&quot;, line 53, in token
    ags_claim['lineitem'] = '{0}/{1}/lineitems/{2}'.format(root_url, course.id, resource_link.lineitem.id)
AttributeError: 'LineItem' object has no attribute 'id'</textarea>
  </form>
</div>
<div class="explanation">
  The debugger caught an exception in your WSGI application.  You can now
  look at the traceback which led to the error.  <span class="nojavascript">
  If you enable JavaScript you can also use additional features such as code
  execution (if the evalex feature is enabled), automatic pasting of the
  exceptions and much more.</span>
</div>
      <div class="footer">
        Brought to you by <strong class="arthur">DON'T PANIC</strong>, your
        friendly Werkzeug powered traceback interpreter.
      </div>
    </div>

    <div class="pin-prompt">
      <div class="inner">
        <h3>Console Locked</h3>
        <p>
          The console is locked and needs to be unlocked by entering the PIN.
          You can find the PIN printed out on the standard output of your
          shell that runs the server.
        <form>
          <p>PIN:
            <input type=text name=pin size=14>
            <input type=submit name=btn value="Confirm Pin">
        </form>
      </div>
    </div>
  </body>
</html>

<!--

Traceback (most recent call last):
  File "/home/claude/.py3envs/lti_bootcamp/lib/python3.5/site-packages/flask/app.py", line 1997, in __call__
    return self.wsgi_app(environ, start_response)
  File "/home/claude/.py3envs/lti_bootcamp/lib/python3.5/site-packages/flask/app.py", line 1985, in wsgi_app
    response = self.handle_exception(e)
  File "/home/claude/.py3envs/lti_bootcamp/lib/python3.5/site-packages/flask/app.py", line 1540, in handle_exception
    reraise(exc_type, exc_value, tb)
  File "/home/claude/.py3envs/lti_bootcamp/lib/python3.5/site-packages/flask/_compat.py", line 33, in reraise
    raise value
  File "/home/claude/.py3envs/lti_bootcamp/lib/python3.5/site-packages/flask/app.py", line 1982, in wsgi_app
    response = self.full_dispatch_request()
  File "/home/claude/.py3envs/lti_bootcamp/lib/python3.5/site-packages/flask/app.py", line 1614, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/home/claude/.py3envs/lti_bootcamp/lib/python3.5/site-packages/flask/app.py", line 1517, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/home/claude/.py3envs/lti_bootcamp/lib/python3.5/site-packages/flask/_compat.py", line 33, in reraise
    raise value
  File "/home/claude/.py3envs/lti_bootcamp/lib/python3.5/site-packages/flask/app.py", line 1612, in full_dispatch_request
    rv = self.dispatch_request()
  File "/home/claude/.py3envs/lti_bootcamp/lib/python3.5/site-packages/flask/app.py", line 1598, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "/home/claude/projects/lti_bootcamp/server/lti_platform.py", line 77, in student_launch
    resource_link=resource_link)
  File "/home/claude/projects/lti_bootcamp/server/ltiplatform/ltiplatform_manager.py", line 53, in token
    ags_claim['lineitem'] = '{0}/{1}/lineitems/{2}'.format(root_url, course.id, resource_link.lineitem.id)
AttributeError: 'LineItem' object has no attribute 'id'

-->
`

### Task 1: Decode the launch

Now, same as with the Deep Linking request, we decode the token:

In [149]:
encoded_jwt = post_data['id_token']
jwt_headers = jwt.get_unverified_header(encoded_jwt)

student_launch = jwt.decode(encoded_jwt, 
                            platform_keys[jwt_headers['kid']], 
                            jwt_headers['alg'],
                            audience = tool_info['client_id'])

DecodeError: Invalid header padding

## Task 2: extract information to show the correct activity

The launch gives information about the user, her role, the course, but also which actual resource we want to launch into.

In [100]:
# fc(claim) prefix the claim with http://imsglobal.org/lti/
mdb('1. Is this a resource link launch? {0}'.format(student_launch[fc('message_type')] == 'LTIResourceLinkLaunch'))
mdb('1. What is the id of the resource link that is launched? {0}'.format(student_launch[fc('resource_link')]['id']))
mdb('1. What is the name of the resource that is launched? {0}'.format(student_launch[fc('resource_link')]['title']))
mdb('1. What is the current course id? {0}'.format(student_launch[fc('context')]['id']))
mdb('1. What is the current user id? {0}'.format(student_launch['sub']))
is_learner = len(list(filter(lambda x: 'learner' in x.lower(), student_launch[fc('roles')])))>0
mdb('1. Is this user a student? {0}'.format(is_learner))
return_url = student_launch[fc('launch_presentation')]['return_url']
mdb('1. Where should I redirect the browser too when done? {0}'.format(return_url))
mdb('1. Which lab should be displayed? {0}'.format(student_launch[fc('custom')]['lab']))
mdb()


1. Is this a resource link launch? True
1. What is the id of the resource link that is launched? 435a6991-11db-11e8-9469-c48e8ffb7857
1. What is the name of the resource that is launched? An assignment
1. What is the current course id? 3fdbf54a-11db-11e8-9469-c48e8ffb7857
1. What is the current user id? LTIBCU_7
1. Is this user a student? True
1. Where should I redirect the browser too when done? http://localhost:5000/
1. Which lab should be displayed? sim3a